# M1 Asking LLMs for Structured Data

[Better Data Extraction Using Pydantic and OpenAI Function Calls](https://wandb.ai/jxnlco/function-calls/reports/Better-Data-Extraction-Using-Pydantic-and-OpenAI-Function-Calls--Vmlldzo0ODU4OTA3?utm_source=course&utm_medium=course&utm_campaign=jason)


[1-Issues with JSON and dictionaries notebook](https://github.com/wandb/edu/blob/main/llm-structured-extraction/1.introduction.ipynb)
  *   Validar os dicionarios para saber o tipo dos dados das keys e valores
  *   Solicitar json corretamente no prompt das LLMs (Function calling)
      * passar um objeto com o seu schema (PersonBirthday(BaseModel))
      * criar schema do objeto usando PersonBirthday.model_json_schema()
      * [Building a Virtual Assistant with Google Gemini Function Calling](https://wandb.ai/byyoung3/ml-news/reports/Building-a-Virtual-Assistant-with-Google-Gemini-Function-Calling--Vmlldzo2MzE1NTY1?utm_source=course&utm_medium=course&utm_campaign=jason)
      * [Using LLMs to Extract Structured Data: OpenAI Function Calling in Action](https://wandb.ai/darek/llmapps/reports/Using-LLMs-to-Extract-Structured-Data-OpenAI-Function-Calling-in-Action--Vmlldzo0Nzc0MzQ3?utm_source=course&utm_medium=course&utm_campaign=jason)

### The core idea around the Instructor:
* Using function calling allows us use a llm that is finetuned to use json_schema and output json.
* Pydantic can be used to define the object, schema, and validation in one single class, allow us to encapsulate everything neatly.

Why Instructor?

Libraries like Marvin, Langchain, and Llamaindex all now leverage the Pydantic object in similar ways. The goal is to be as light weight as possible, get you as close as possible to the openai api, and then get out of your way.

More importantly, we've also added straight forward validation and reasking to the mix.

The goal of instructor is to show you how to think about structured prompting and provide examples and documentation that you can take with you to any framework.








In [ ]:
# exremplo de function calling
import datetime


class PersonBirthday(BaseModel):
    name: str
    age: int
    birthday: datetime.date


schema = {
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer"},
        "birthday": {"type": "string", "format": "YYYY-MM-DD"},
    },
    "required": ["name", "age"],
    "type": "object",
}

resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"Extract `Jason Liu is thirty years old his birthday is yesturday` into json today is {datetime.date.today()}",
        },
    ],
    functions=[{"name": "Person", "parameters": schema}],
    function_call="auto",
)

PersonBirthday.model_validate_json(resp.choices[0].message.function_call.arguments)

In [ ]:
# exemplo usando instructor
import instructor
import datetime

# patch the client to add `response_model` to the `create` method
client = instructor.patch(OpenAI(), mode=instructor.Mode.MD_JSON)

resp = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {
            "role": "user",
            "content": f"""
            Today is {datetime.date.today()}

            Extract `Jason Liu is thirty years old his birthday is yesturday`
            he lives at 123 Main St, San Francisco, CA""",
        },
    ],
    response_model=PersonAddress,
)
resp

# Prompting LLMs

## classification
For classification, we've found there are generally two methods of modeling:

* using Enums
* using Literals

Use an enum in Python when you need a set of named constants that are related and you want to ensure type safety, readability, and prevent invalid values. Enums are helpful for grouping and iterating over these constants.

Use literals when you have a small, unchanging set of values that you don't need to group or iterate over, and when type safety and preventing invalid values is less of a concern. Literals are simpler and more direct for basic, one-off values.

In [ ]:
import instructor
from openai import OpenAI

from enum import Enum
from pydantic import BaseModel, Field
from typing_extensions import Literal


client = instructor.patch(OpenAI())


# Tip: Do not use auto() as they cast to 1,2,3,4
class House(Enum):
    Gryffindor = "gryffindor"
    Hufflepuff = "hufflepuff"
    Ravenclaw = "ravenclaw"
    Slytherin = "slytherin"


class Character(BaseModel):
    age: int
    name: str
    house: House

    def say_hello(self):
        print(
            f"Hello, I'm {self.name}, I'm {self.age} years old and I'm from {self.house.value.title()}"
        )


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

In [ ]:
class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

## Arbitrary properties
Often times there are long properties that you might want to extract from data that we can not specify in advance. We can get around this by defining an arbitrary key value store like so:

In [ ]:
from typing import List


class Property(BaseModel):
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

### Limiting the length of lists
In later chapters we'll talk about how to use validators to assert the length of lists but we can also use prompting tricks to enumerate values. Here we'll define an index to count the properties.

In the following example instead of extraction we're going to work on generation instead.

In [ ]:
class Property(BaseModel):
    index: str = Field(..., description="Monotonically increasing ID")
    key: str = Field(description="Must be snake case")
    value: str


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]
    properties: List[Property] = Field(
        ...,
        description="Numbered list of arbitrary extracted properties, should be exactly 5",
    )


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Snape from Harry Potter"}],
    response_model=Character,
)
resp.model_dump()

## Defining Multiple Entities
Now that we see a single entity with many properties we can continue to nest them into many users! If we add the Iterable type to the User type we can define multiple users in a single prompt, now instead of extracting one user we can extract many users. But only after the completion of the prompt.

In [ ]:
from typing import Iterable


class Character(BaseModel):
    age: int
    name: str
    house: Literal["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "Five characters from Harry Potter"}],
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

## Defining Relationships
Now only can we define lists of users, with list of properties one of the more interesting things I've learned about prompting is that we can also easily define lists of references.

With the tools we've discussed, we can find numerous real-world applications in production settings. These include extracting action items from transcripts, generating fake data, filling out forms, and creating objects that correspond to generative UI. These simple tricks will be highly useful.

In [ ]:
class Character(BaseModel):
    id: int
    name: str
    friends_array: List[int] = Field(description="Relationships to their friends using the id")


resp = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=[{"role": "user", "content": "5 kids from Harry Potter"}],
    stream=True,
    response_model=Iterable[Character],
)

for character in resp:
    print(character)

## Missing Data
The Maybe pattern is a concept in functional programming used for error handling. Instead of raising exceptions or returning None, you can use a Maybe type to encapsulate both the result and potential errors.

This pattern is particularly useful when making LLM calls, as providing language models with an escape hatch can effectively reduce hallucinations.

In [ ]:
from typing import Optional

class Character(BaseModel):
    age: int
    name: str

class MaybeCharacter(BaseModel):
    result: Optional[Character] = Field(default=None)
    error: bool = Field(default=False)
    message: Optional[str]

@weave.op()
def extract(content: str) -> MaybeCharacter:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=MaybeCharacter,
        messages=[
            {"role": "user", "content": f"Extract `{content}`"},
        ],
    )

extract("Harry Potter")

user = extract("404 Error")

if user.error:
    raise ValueError(user.message)

# Applying Structured Output to RAG applications

## Example 1) Improving Extractions
A common method of using structured output is to extract information from a document and use it to answer a question. Directly, we can be creative in how we extract, summarize and generate potential questions in order for our embeddings to do better.

For example, instead of using just a text chunk we could try to:

* extract key words and themes
* extract hypothetical questions
* generate a summary of the text

In the example below, we use the instructor library to extract the key words and themes from a text chunk and use them to answer a question.

In [ ]:
class Extraction(BaseModel):
    topic: str
    summary: str
    hypothetical_questions: List[str] = Field(
        default_factory=list,
        description="Hypothetical questions that this document could answer",
    )
    keywords: List[str] = Field(
        default_factory=list, description="Keywords that this document is about"
    )

from pprint import pprint
from typing import Iterable


text_chunk = """
## Simple RAG

**What is it?**

The simplest implementation of RAG embeds a user query and do a single embedding search in a vector database, like a vector store of Wikipedia articles. However, this approach often falls short when dealing with complex queries and diverse data sources.

**What are the limitations?**

- **Query-Document Mismatch:** It assumes that the query and document embeddings will align in the vector space, which is often not the case.
    - Query: "Tell me about climate change effects on marine life."
    - Issue: The model might retrieve documents related to general climate change or marine life, missing the specific intersection of both topics.
- **Monolithic Search Backend:** It relies on a single search method and backend, reducing flexibility and the ability to handle multiple data sources.
    - Query: "Latest research in quantum computing."
    - Issue: The model might only search in a general science database, missing out on specialized quantum computing resources.
- **Text Search Limitations:** The model is restricted to simple text queries without the nuances of advanced search features.
    - Query: "what problems did we fix last week"
    - Issue: cannot be answered by a simple text search since documents that contain problem, last week are going to be present at every week.
- **Limited Planning Ability:** It fails to consider additional contextual information that could refine the search results.
    - Query: "Tips for first-time Europe travelers."
    - Issue: The model might provide general travel advice, ignoring the specific context of first-time travelers or European destinations.
"""

extractions = client.chat.completions.create(
    model="gpt-4-1106-preview",
    stream=True,
    response_model=Iterable[Extraction],
    messages=[
        {
            "role": "system",
            "content": "Your role is to extract chunks from the following and create a set of topics.",
        },
        {"role": "user", "content": text_chunk},
    ],
)


for extraction in extractions:
    pprint(extraction.model_dump())

## Example 2) Understanding 'recent queries' to add temporal context
One common application of using structured outputs for query understanding is to identify the intent of a user's query. In this example we're going to use a simple schema to seperately process the query to add additional temporal context.

In this example, DateRange and Query are Pydantic models that structure the user's query with a date range and a list of domains to search within.

These models restructure the user's query by including a rewritten query, a range of published dates, and a list of domains to search in.

Using the new restructured query, we can apply this pattern to our function calls to obtain results that are optimized for our backend.

In [ ]:
from datetime import date


class DateRange(BaseModel):
    start: date
    end: date


class Query(BaseModel):
    rewritten_query: str
    published_daterange: DateRange

def expand_query(q) -> Query:
    return client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": f"You're a query understanding system for the Metafor Systems search engine. Today is {date.today()}. Here are some tips: ...",
            },
            {"role": "user", "content": f"query: {q}"},
        ],
    )


query = expand_query("What are some recent developments in AI?")
query

In [ ]:
class DateRange(BaseModel):
    chain_of_thought: str = Field(
        description="Think step by step to plan what is the best time range to search in"
    )
    start: date
    end: date


class Query(BaseModel):
    rewritten_query: str = Field(
        description="Rewrite the query to make it more specific"
    )
    published_daterange: DateRange = Field(
        description="Effective date range to search in"
    )


def expand_query(q) -> Query:
    return client.chat.completions.create(
        model="gpt-4-1106-preview",
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": f"You're a query understanding system for the Metafor Systems search engine. Today is {date.today()}. Here are some tips: ...",
            },
            {"role": "user", "content": f"query: {q}"},
        ],
    )


expand_query("What are some recent developments in AI?")

## Using Weights & Biases to track experiments
While running a function like this in production is quite simple, a lot of time will be spent on iterating and improving the model. To do this, we can use Weights & Biases to track our experiments.

In order to do so we need to manage a few things:

* Save input and output pairs for later
* Save the JSON schema for the response_model
* Having snapshots of the model and data allow us to compare results over time, and as we make changes to the model we can see how the results change.

This is particularly useful when we might want to blend a mix of synthetic and real data to evaluate our model. We can use the wandb library to track our experiments and save the results to a dashboard.

In [ ]:
import json
import instructor

from openai import AsyncOpenAI
from helpers import dicts_to_df
from datetime import date
from pydantic import BaseModel, Field


class DateRange(BaseModel):
    chain_of_thought: str = Field(
        description="Think step by step to plan what is the best time range to search in"
    )
    start: date
    end: date


class Query(BaseModel):
    rewritten_query: str = Field(
        description="Rewrite the query to make it more specific"
    )
    published_daterange: DateRange = Field(
        description="Effective date range to search in"
    )

    def report(self):
        dct = self.model_dump()
        dct["usage"] = self._raw_response.usage.model_dump()
        return dct



# We'll use a different client for async calls
# To highlight the difference and how we can use both
aclient = instructor.patch(AsyncOpenAI())


async def expand_query(
    q, *, model: str = "gpt-4-1106-preview", temp: float = 0
) -> Query:
    return await aclient.chat.completions.create(
        model=model,
        temperature=temp,
        response_model=Query,
        messages=[
            {
                "role": "system",
                "content": f"You're a query understanding system for the Metafor Systems search engine. Today is {date.today()}. Here are some tips: ...",
            },
            {"role": "user", "content": f"query: {q}"},
        ],
    )

In [ ]:
import asyncio
import time
import pandas as pd
import wandb

model = "gpt-4-1106-preview"
temp = 0

run = wandb.init(
    project="llmeng-1-nb3",
    config={"model": model, "temp": temp},
)

test_queries = [
    "latest developments in artificial intelligence last 3 weeks",
    "renewable energy trends past month",
    "quantum computing advancements last 2 months",
    "biotechnology updates last 10 days",
]
start = time.perf_counter()
queries = await asyncio.gather(
    *[expand_query(q, model=model, temp=temp) for q in test_queries]
)
duration = time.perf_counter() - start

with open("schema.json", "w+") as f:
    schema = Query.model_json_schema()
    json.dump(schema, f, indent=2)

with open("results.jsonlines", "w+") as f:
    for query in queries:
        f.write(query.model_dump_json() + "\n")

df = dicts_to_df([q.report() for q in queries])
df["input"] = test_queries
df.to_csv("results.csv")


run.log({"schema": wandb.Table(dataframe=pd.DataFrame([{"schema": schema}]))})

run.log(
    {
        "usage_total_tokens": df["usage_total_tokens"].sum(),
        "usage_completion_tokens": df["usage_completion_tokens"].sum(),
        "usage_prompt_tokens": df["usage_prompt_tokens"].sum(),
        "duration (s)": duration,
        "average duration (s)": duration / len(queries),
        "n_queries": len(queries),
    }
)


run.log(
    {
        "results": wandb.Table(dataframe=df),
    }
)

files = wandb.Artifact("data", type="dataset")

files.add_file("schema.json")
files.add_file("results.jsonlines")
files.add_file("results.csv")


run.log_artifact(files)
run.finish()

## Example 3) Personal Assistants, parallel processing
A personal assistant application needs to interpret vague queries and fetch information from multiple backends, such as emails and calendars. By modeling the assistant's capabilities using Pydantic, we can dispatch the query to the correct backend and retrieve a unified response.

For instance, when you ask, "What's on my schedule today?", the application needs to fetch data from various sources like events, emails, and reminders. This data is stored across different backends, but the goal is to provide a consolidated summary of results.

It's important to note that the data from these sources may not be embedded in a search backend. Instead, they could be accessed through different clients like a calendar or email, spanning both personal and professional accounts.



In [ ]:
from typing import Literal


class SearchClient(BaseModel):
    query: str = Field(description="The search query that will go into the search bar")
    keywords: List[str]
    email: str
    source: Literal["gmail", "calendar"]
    date_range: DateRange


class Retrival(BaseModel):
    queries: List[SearchClient]

Now, we can utilize this with a straightforward query such as "What do I have today?".

The system will attempt to asynchronously dispatch the query to the appropriate backend.

However, it's still crucial to remember that effectively prompting the language model is still a key aspect.

In [ ]:
retrival = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=Retrival,
    messages=[
        {
            "role": "system",
            "content": f"""You are Jason's personal assistant.
                He has two emails jason@work.com jason@personal.com
                Today is {date.today()}""",
        },
        {"role": "user", "content": "What do I have today for work? any new emails?"},
    ],
)
print(retrival.model_dump_json(indent=4))

To make it more challenging, we will assign it multiple tasks, followed by a list of queries that are routed to various search backends, such as email and calendar. Not only do we dispatch to different backends, over which we have no control, but we are also likely to render them to the user in different ways.

In [ ]:
retrival = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_model=Retrival,
    messages=[
        {
            "role": "system",
            "content": f"""You are Jason's personal assistant.
                He has two emails jason@work.com jason@personal.com
                Today is {date.today()}""",
        },
        {
            "role": "user",
            "content": "What meetings do I have today and are there any important emails I should be aware of",
        },
    ],
)
print(retrival.model_dump_json(indent=4))

## Example 4) Decomposing questions
Lastly, a lightly more complex example of a problem that can be solved with structured output is decomposing questions. Where you ultimately want to decompose a question into a series of sub-questions that can be answered by a search backend. For example

"Whats the difference in populations of jason's home country and canada?"

You'd ultimately need to know a few things

* Jason's home country
* The population of Jason's home country
* The population of Canada
* The difference between the two

This would not be done correctly as a single query, nor would it be done in parallel, however there are some opportunities try to be parallel since not all of the sub-questions are dependent on each other.

In [ ]:
class Question(BaseModel):
    id: int = Field(..., description="A unique identifier for the question")
    query: str = Field(..., description="The question decomposited as much as possible")
    subquestions: List[int] = Field(
        default_factory=list,
        description="The subquestions that this question is composed of",
    )


class QueryPlan(BaseModel):
    root_question: str = Field(..., description="The root question that the user asked")
    plan: List[Question] = Field(
        ..., description="The plan to answer the root question and its subquestions"
    )


retrival = client.chat.completions.create(
    model="gpt-4-1106-preview",
    response_model=QueryPlan,
    messages=[
        {
            "role": "system",
            "content": "You are a query understanding system capable of decomposing a question into subquestions.",
        },
        {
            "role": "user",
            "content": "What is the difference between the population of jason's home country and canada?",
        },
    ],
)

print(retrival.model_dump_json(indent=4))

# Understanding Validators and controlling responses

## Previously we went over how to use structured Extraction to Query and Plan a search request

In this section we'll aim to

Expand on how Pydantic's validation features work
Apply them to generate better responses by using feedback and validation.
Pydantic offers a customizable and expressive validation framework for Python. Instructor leverages Pydantic's validation framework to provide a uniform developer experience for both code-based and LLM-based validation, as well as a reasking mechanism for correcting LLM outputs based on validation errors. To learn more check out the Pydantic docs on validators.

Validators will enable us to control outputs by defining a function like so:

def validation_function(value):
    if condition(value):
        raise ValueError("Value is not valid")
    return mutation(value)
Before we get started lets go over the general shape of a validator:

In [1]:
#Defining Validator Functions
from typing_extensions import Annotated
from pydantic import BaseModel, AfterValidator, WithJsonSchema


def name_must_contain_space(v: str) -> str:
    if " " not in v:
        raise ValueError("Name must contain a space.")
    return v

def uppercase_name(v: str) -> str:
    return v.upper()

FullName = Annotated[
    str,
    AfterValidator(name_must_contain_space),
    AfterValidator(uppercase_name),
    WithJsonSchema(
        {
            "type": "string",
            "description": "The user's full name",
        }
    )]

class UserDetail(BaseModel):
    age: int
    name: FullName
UserDetail(age=30, name="Jason Liu")
UserDetail.model_json_schema()
try:
    person = UserDetail.model_validate({"age": 24, "name": "Jason"})
except Exception as e:
    print(e)


# Using Field
We can also use the Field class to define validators. This is useful when we want to define a validator for a field that is primitive, like a string or integer which supports a limited number of validators.

from pydantic import Field


Age = Annotated[int, Field(gt=0)]

class UserDetail(BaseModel):
    age: Age
    name: FullName

try:
    person = UserDetail(age=-10, name="Jason")
except Exception as e:
    print(e)

SyntaxError: invalid syntax (<ipython-input-1-585039876673>, line 36)

In [ ]:
# Providing Context
from pydantic import ValidationInfo

def message_cannot_have_blacklisted_words(v: str, info: ValidationInfo) -> str:
    blacklist = info.context.get("blacklist", [])
    for word in blacklist:
        assert word not in v.lower(), f"`{word}` was found in the message `{v}`"
    return v

ModeratedStr = Annotated[str, AfterValidator(message_cannot_have_blacklisted_words)]

class Response(BaseModel):
    message: ModeratedStr


try:
    Response.model_validate(
        {"message": "I will hurt them."},
        context={
            "blacklist": {
                "rob",
                "steal",
                "kill",
                "attack",
            }
        },
    )
except Exception as e:
    print(e)

#Using OpenAI Moderation
#To enhance our validation measures, we'll extend the scope to flag any answer that contains hateful content, harassment, or similar issues. OpenAI offers a moderation endpoint that addresses these concerns, and it's freely available when using OpenAI models.

#With the instructor library, this is just one function edit away:

from typing import Annotated
from pydantic import AfterValidator
from instructor import openai_moderation

import instructor
from openai import OpenAI

client = instructor.patch(OpenAI())

# This uses Annotated which is a new feature in Python 3.9
# To define custom metadata for a type hint.
ModeratedStr = Annotated[str, AfterValidator(openai_moderation(client=client))]


class Response(BaseModel):
    message: ModeratedStr


try:
    Response(message="I want to make them suffer the consequences")
except Exception as e:
    print(e)



# General Validator
from instructor import llm_validator

HealthTopicStr = Annotated[
    str,
    AfterValidator(
        llm_validator(
            "don't talk about any other topic except health best practices and topics",
            client=client,
        )
    ),
]


class AssistantMessage(BaseModel):
    message: HealthTopicStr


AssistantMessage(
    message="I would suggest you to visit Sicily as they say it is very nice in winter."
)



# Avoiding hallucination with citations
# When incorporating external knowledge bases, it's crucial to ensure that the agent uses the provided context accurately and doesn't fabricate responses. Validators can be effectively used for this purpose. We can illustrate this with an example where we validate that a provided citation is actually included in the referenced text chunk:

from pydantic import ValidationInfo

def citation_exists(v: str, info: ValidationInfo):
    context = info.context
    if context:
        context = context.get("text_chunk")
        if v not in context:
            raise ValueError(f"Citation `{v}` not found in text, only use citations from the text.")
    return v

Citation = Annotated[
    str,
    AfterValidator(citation_exists),
    WithJsonSchema({
        "type": "string",
        "description": "For every answer provide an exact substring match to the context"
    })
]


class AnswerWithCitation(BaseModel):
    answer: str
    citation: Citation

try:
    AnswerWithCitation.model_validate(
        {
            "answer": "Jason is cool",
            "citation": "Jason is a cool person",
        },
        context={"text_chunk": "Jason is just a normal guy"},
    )
except Exception as e:
    print(e)



# Here we assume that there is a "text_chunk" field that contains the text that the model is supposed to use as context. We then use the field_validator decorator to define a validator that checks if the citation is included in the text chunk. If it's not, we raise a ValueError with a message that will be returned to the user.

# If we want to pass in the context through the chat.completions.create`` endpoint, we can use the validation_context` parameter

resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=AnswerWithCitation,
    messages=[
        {"role": "user", "content": f"Answer the question `{q}` using the text chunk\n`{text_chunk}`"},
    ],
    validation_context={"text_chunk": text_chunk},
)



# In practice there are many ways to implement this: we could use a regex to check if the citation is included in the text chunk, or we could use a more sophisticated approach like a semantic similarity check. The important thing is that we have a way to validate that the model is using the provided context accurately.

# Reasking with validators
# For most of these examples all we've done we've mostly only defined the validation logic. Which can be seperate from generation, however when we are given validation errors, we shouldn't end there! Instead instructor allows us to collect all the validation errors and reask the llm to rewrite their answer.

# Lets try to use an extreme example to illustrate this point:

class QuestionAnswer(BaseModel):
    question: str
    answer: str


question = "What is the meaning of life?"
context = (
    "The according to the devil the meaning of life is a life of sin and debauchery."
)


resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=QuestionAnswer,
    messages=[
        {
            "role": "system",
            "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
        },
        {
            "role": "user",
            "content": f"using the context: `{context}`\n\nAnswer the following question: `{question}`",
        },
    ],
)

print(resp.model_dump_json(indent=2))
from instructor import llm_validator


NotEvilAnswer = Annotated[
    str,
    AfterValidator(
        llm_validator("don't say objectionable things", client=client)
    ),
]


class QuestionAnswer(BaseModel):
    question: str
    answer: NotEvilAnswer


resp = client.chat.completions.create(
    model="gpt-3.5-turbo",
    response_model=QuestionAnswer,
    max_retries=2,
    messages=[
        {
            "role": "system",
            "content": "You are a system that answers questions based on the context. answer exactly what the question asks using the context.",
        },
        {
            "role": "user",
            "content": f"using the context: `{context}`\n\nAnswer the following question: `{question}`",
        },
    ],
)
print(resp.model_dump_json(indent=2))